In [12]:
import numpy as np
import pandas as pd

# Question 1

In [106]:

# you can use this table as an example
distr_table = pd.DataFrame({
    'X': [0, 0, 1, 1],
    'Y': [1, 2, 1, 2],
    'pr': [0.3, 0.25, 0.15, 0.3]
})

class CheckIndependence:

    def __init__(self):
        self.version = 1

    def check_independence(self, distr_table: pd.DataFrame):
        X = list(distr_table['X'])
        Y = list(distr_table['Y'])
        p = list(distr_table['pr'])

        marginal_x = {}
        marginal_y = {}
        
        for i in range(len(distr_table)):
            if X[i] not in marginal_x:
                marginal_x[X[i]] = p[i]
            else:
                marginal_x[X[i]] += p[i]
            
            if Y[i] not in marginal_y:
                marginal_y[Y[i]] = p[i]
            else:
                marginal_y[Y[i]] += p[i]

        print(marginal_x, marginal_y)
        
        are_independent = True
        
        
        Exy = 0
        
        
        for x in X:
            for y in Y:
                pxy = distr_table.loc[(distr_table["X"] == x) & (distr_table["Y"] == y)]["pr"].iloc[0]
                if marginal_x[x]*marginal_y[y] != pxy:
                    are_independent = False
                Exy += x*y*pxy
    
        Ex = 0
        Ey = 0
        Exy = 0
        Ex2 = 0
        Ey2 = 0

        for x in marginal_x:
            Ex += x*marginal_x[x]
            Ex2 += x*x*marginal_x[x]
            
        for y in marginal_y:
            Ey += y*marginal_y[y]
            Ey2 += y*y*marginal_y[y]

        cov = Exy - Ex*Ey
        stdx = np.sqrt(Ex2-Ex*Ex)
        stdy = np.sqrt(Ey2-Ey*Ey)
        corr = cov/(stdx*stdy)
        
        
        ans = {"are_independent": are_independent, "cov": cov, 
               "corr": corr}
        
        print(Ex, Ey, Ex2, Ey2,Exy)
        
        return ans
    
    


In [107]:
ci = CheckIndependence()

ci.check_independence(distr_table)

{0: 0.55, 1: 0.44999999999999996} {1: 0.44999999999999996, 2: 0.55}
0.44999999999999996 1.55 0.44999999999999996 2.6500000000000004 0


{'are_independent': False,
 'cov': -0.6974999999999999,
 'corr': -2.8181818181818175}

# Question 2

In [108]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

def run_clf(data_train, data_test, n):
    
    output = {}

    # taking data
    y_train = data_train["target"]
    y_test = data_test["target"]

    X_train = data_train.drop("target", axis = 1)
    X_test = data_test.drop("target", axis = 1)


    clf = DecisionTreeClassifier(random_state=0)
    
    clf.fit(X_train, y_train)
    
    score_orig = clf.score(X_test, y_test)
    
    print(f"test acc: {score_orig}")
    
    explanatory_features = X_train.columns.tolist()
    
    importances = {}
    
    for col in explanatory_features:
        # Creating a copy of X_train we can modify at the per variable level
        temp_X_train = X_train.copy()
        
        temp_X_train[col] = np.random.permutation(X_train[col])
        
        clf_temp = DecisionTreeClassifier(random_state=0)
        
        clf_temp.fit(temp_X_train, y_train)
        
        score_x = clf_temp.score(X_test, y_test)

        importances[col] = score_orig - score_x
    
    
    # sort
    importances = dict(sorted(importances.items(), key=lambda item: item[1]))
    
    # we don't do reverse = true here because the most important feature 
    # will have a negative number that means higher test accuracy
    # since its score_orig - score_x = (negative number here means)
    # score_x is higher
    
    importances_tuple_list = [(i,j) for i, j in importances.items()]
        
    
    output["most_important"] = importances_tuple_list
    
    # calculating false positive predictions
    
    def evaluate():
    
        scores = {}
        
        y_pred = clf.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
    
        return cm
    
    
    cm = evaluate()
    
    def evaluate_with_n():
        
        feature_names = [i[0] for i in importances_tuple_list]
        
        train_with = [feature_names[i] for i in range(n)]
        
        
        clf_3 = DecisionTreeClassifier(random_state=0)
        
        temp_X_train_3 = X_train.copy()[train_with]
        
        clf_3.fit(temp_X_train_3, y_train)
        
        score_x = clf_3.score(X_test[train_with], y_test)
        
        y_pred = clf_3.predict(X_test[train_with])
        cm = confusion_matrix(y_test, y_pred)
    
        return cm
    
    cm_2 = evaluate_with_n()
    
    
    output['fp'] = cm[1][0]
    output["fp_most_important"] = cm_2[1][0]
    
    return output

In [109]:
# data ingestion
data = [pd.read_csv('data/{}/{}.csv'.format(file, df)) for file in ['credit_data', 'gender_voice'] for df in ['train', 'test']]
credit_train = data[0]
credit_test = data[1]
gender_train = data[2]
gender_test = data[3]

credit_train.head()

,duration,credit_amount,installment_rate,present_residence,age,existing_credits,dependents,checking_account_status_A12,checking_account_status_A13,checking_account_status_A14,...,other_installment_plans_A142,other_installment_plans_A143,housing_A152,housing_A153,job_A172,job_A173,job_A174,telephone_A192,foreign_worker_A202,target
0,36,8086.0,2.0,4.0,42.0,4.0,1,1,0,0,...,0,1,1,0,0,0,1,1,0,1
1,15,3812.0,1.0,4.0,23.0,1.0,1,0,0,1,...,0,1,1,0,0,1,0,1,0,0
2,36,2145.0,2.0,1.0,24.0,2.0,1,0,0,0,...,0,1,1,0,0,1,0,1,0,1
3,12,2578.0,3.0,4.0,55.0,1.0,1,0,0,0,...,0,1,0,1,0,0,1,0,0,0
4,21,5003.0,1.0,4.0,29.0,2.0,1,0,0,1,...,0,0,1,0,0,1,0,1,0,1


In [110]:
# unit testing permutation feature
a = np.array(credit_train['duration'])

# showing just the head
np.random.permutation(np.array(a))[:10]

array([12, 21, 12, 21, 12,  6, 15, 24,  9, 18])

In [111]:
# test
run_clf(credit_train, credit_test, 3)

test acc: 0.69


{'most_important': [('checking_account_status_A13', -0.040000000000000036),
  ('purpose_A42', -0.030000000000000027),
  ('credit_history_A32', -0.020000000000000018),
  ('purpose_A43', -0.020000000000000018),
  ('age', -0.010000000000000009),
  ('credit_history_A31', -0.010000000000000009),
  ('present_employment_A72', -0.010000000000000009),
  ('job_A174', -0.010000000000000009),
  ('installment_rate', 0.0),
  ('purpose_A46', 0.0),
  ('savings_A65', 0.0),
  ('other_installment_plans_A142', 0.0),
  ('housing_A153', 0.0),
  ('purpose_A45', 0.009999999999999898),
  ('purpose_A49', 0.009999999999999898),
  ('savings_A63', 0.009999999999999898),
  ('other_debtors_A102', 0.009999999999999898),
  ('property_A122', 0.009999999999999898),
  ('job_A172', 0.009999999999999898),
  ('telephone_A192', 0.009999999999999898),
  ('duration', 0.019999999999999907),
  ('purpose_A44', 0.019999999999999907),
  ('purpose_A48', 0.019999999999999907),
  ('present_employment_A74', 0.019999999999999907),
  ('h

# Question 3

The effects of global warming are become more and more noticable each year. That is why many contries aim to reduce greenhouse gas emissions by generating electricity from the wind instead of from fossil suels. Wind turbines have fewer negative effects on the environment and emit less carbon dioxide.

A company has got a contract to build a wind farm on an enclosed, rectangular area of land. The goal is to build as many turbines as possible, to maximize the wind farm's power capacity.

It was recently discovered that the land is inhabited by an endangered species of bird. Thus, the company wants to build the wind turbines at a distance greater than K from every habitat.

The map of the land is given as rectangular matrix A of integers, where birds' habitats are represented by 1 and all other locations are represented by 0.

Once constructed, a turbine occupies exactly one cell of the map and each cell can contain at most one turbine. The distance between the two locations is measured as the sum of horizontal and vertical distances between two cells on the map. For example, the distance between cells $$A[3][2]$$ and $$A[1][4]$$ is $$\lvert1+3\rvert+\lvert4-2\rvert=4$$.

Write a function:

    def solution(A,k):
    
that given a matrix A consistent of N rows and M columns and an integer K, returns the maximum number of wind turbines that can be build white following the environmentalist' advice.




Write an efficient algorithm for the following assumptions:

* N and M are integers within the range [1..600]
* K is an integer within the range [1..1200]
* each element of matrix A is an integer that can have one of the following values: 0,1




In [112]:
import numpy as np

In [154]:
def solution(A,k):
    ''' Writing a function that implements environmentalists advice
    
    Parameters:
    -----------
    A: n x m matrix
    k: distance requirement
    
    Returns:
    --------
    max_turbines: maximum number of turbines that can be built using 
        the environmentalist's advice
    '''
    
    # locations of all the birds
    birds = np.asarray(np.where(A == 1)).T
    
    for bird_loc in birds:
        # this gives the location of the bird in the format:
        # np.array([x, y])
        
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                # using the distance metric defined in the question
                distance = abs(j - bird_loc[1]) + abs(i - bird_loc[0])
                
                # generalizing this condition for every k    
                if (distance >0) & (distance <=k):
                    # the integer -10 will indicate where we cannot
                    # place a wind turbine
                    A[i][j] = -10
                elif distance == 0:
                    # this is where a bird is located
                    A[i][j] = -50
                    
    print(f"\nbird locations: {birds}")
    
    print("\nWhat the matrix looks like after I run my function\
          \n    (-50 means bird, -10 means cannot place wind turbine here):\n")
    print(A)
    
    # actually returning the number of places you can place a wind turbine
    
    # this is the number of squares
    availibility = A.shape[0] * A.shape[1]
    
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            # if it is a bird or in a forbidden area
            if (A[i][j] == -10) | (A[i][j] == -50):
                availibility -= 1
    
    return availibility

In [155]:
# this last question I didn't have time to save the 
# sample code in Codility, so I am using my own implementation

# my test case

def create_A(n = 3, m = 4, k = 1, birds = [(0,0), (2,2), (1,3)]):
    ''' This function is to make the A matrix for my test case 
    with the birds located with a 1 at the coordinates specified
    with the birds list of tuples.
    '''
    
    A = np.full((n,m), 0)

    # let's put a bird at all the places in the list

    for bird in birds:
        A[bird[0]][bird[1]] = 1
        
    return A

In [156]:
A = create_A()

# first test case
print("Matrix A input:")
print(A)

# solution
solution(A, 1)

Matrix A input:
[[1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]]

bird locations: [[0 0]
 [1 3]
 [2 2]]

What the matrix looks like after I run my function          
    (-50 means bird, -10 means cannot place wind turbine here):

[[-50 -10   0 -10]
 [-10   0 -10 -50]
 [  0 -10 -50 -10]]


3

In [157]:
# Now let's see if this works for the bigger matrix AND k value

n = 8
m = 8
k = 2

birds = [(2,2), (6,6)]

A = create_A(n, m, k, birds)

# starting point
print("Matrix A input:")
print(A)

solution(A, k)

Matrix A input:
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]

bird locations: [[2 2]
 [6 6]]

What the matrix looks like after I run my function          
    (-50 means bird, -10 means cannot place wind turbine here):

[[  0   0 -10   0   0   0   0   0]
 [  0 -10 -10 -10   0   0   0   0]
 [-10 -10 -50 -10 -10   0   0   0]
 [  0 -10 -10 -10   0   0   0   0]
 [  0   0 -10   0   0   0 -10   0]
 [  0   0   0   0   0 -10 -10 -10]
 [  0   0   0   0 -10 -10 -50 -10]
 [  0   0   0   0   0 -10 -10 -10]]


40

Great! As you can see my solution works for both test cases.

Now for another test case where there are no birds at all. This should fill everything.

In [158]:
# Solving for no birds test case

n = 3
m = 2
k = 1

birds = []

A = create_A(n, m, k, birds)

# starting point
print("Matrix A input:")
print(A)

solution(A, k)

Matrix A input:
[[0 0]
 [0 0]
 [0 0]]

bird locations: []

What the matrix looks like after I run my function          
    (-50 means bird, -10 means cannot place wind turbine here):

[[0 0]
 [0 0]
 [0 0]]


6

In [159]:
# Out of curiosity, I want to see what K=3 looks like
n = 10
m = 10
k = 3

birds = [(4,4)]

A = create_A(n, m, k, birds)

# starting point
print("Matrix A input:")
print(A)

solution(A, k)

Matrix A input:
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

bird locations: [[4 4]]

What the matrix looks like after I run my function          
    (-50 means bird, -10 means cannot place wind turbine here):

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0 -10   0   0   0   0   0]
 [  0   0   0 -10 -10 -10   0   0   0   0]
 [  0   0 -10 -10 -10 -10 -10   0   0   0]
 [  0 -10 -10 -10 -50 -10 -10 -10   0   0]
 [  0   0 -10 -10 -10 -10 -10   0   0   0]
 [  0   0   0 -10 -10 -10   0   0   0   0]
 [  0   0   0   0 -10   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]


75

Great this works! This should generalize to all test cases.